# Zero Matrix – 2D

If a MxN matrix contains any zero values, set the values of the corresponding row and column to zero as well. First versions work with Python built-ins alone, then I try one with Numpy.

**Bonus:** make it adapt to N-dimensional matrix (in following notebook).

In [2]:
import numpy as np
import random
import timeit
from functools import partial
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib inline

### Make It Work

In [16]:
def zero_matrix(arr_in, validate=True):
    """First version for 2D matrix"""
    
    # edges, checks
    assert len(arr_in) > 0, "Zero length matrix"
    # validate evenness of n-dimension
    if validate:
        n_dim = len(arr_in[0])
        for row in arr_in[1:]:
            assert len(row) == n_dim, "One or more rows are of uneven lengths"
    
    # compile list of zero-coordinates
    zeroes = []
    for m_ix, row in enumerate(arr_in):
        for n_ix, val in enumerate(row):
            if val == 0:
                zeroes.append((m_ix, n_ix))
    
    # zero out corresponding row/col for each zero value
    if len(zeroes) == 0:
        return arr_in
    else:
        for m_ix, n_ix in zeroes:
            # set column to zeroes
            for m_ix_dyn, _ in enumerate(arr_in):
                arr_in[m_ix_dyn][n_ix] = 0
            # set row to zeroes
            arr_in[m_ix] = [0 for _ in range(len(arr_in[0]))]
    
    return arr_in

In [90]:
def gen_test_arr(p_zero, m, n=None):
    """Generates MxN array filled with randomly assigned 0s and 1s"""

    if not n:
        n = m
    
    test_arr = [
        random.choices(range(2), weights=[p_zero, 1-p_zero], k=n)
        for _ in range(m)
    ]
        
    return test_arr

In [93]:
test_arr = gen_test_arr(0.05, 5, 8)
test_arr

[[1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1]]

In [94]:
zero_matrix(test_arr)

[[0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 1, 0, 1]]

In [95]:
del test_arr[3][5]

In [96]:
zero_matrix(test_arr)

AssertionError: One or more rows are of uneven lengths

It works, and breaks when it should. But how much faster would it be to build a list of zero coordinates with a list comprehension instead?

### Gathering Speed

In [5]:
def coords_by_loops(arr_in):
    zeroes = []
    for m_ix, row in enumerate(arr_in):
        for n_ix, val in enumerate(row):
            if val == 0:
                zeroes.append((m_ix, n_ix))
    return zeroes
                
def coords_by_list_comp(arr_in):
    return [(m,n) for m, row in enumerate(arr_in) 
            for n, v in enumerate(row) 
            if v == 0
           ]

In [189]:
test_arr = gen_test_arr(0.05, 10)

In [190]:
%%timeit
coords_by_loops(test_arr)

11.6 µs ± 610 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [191]:
%%timeit
coords_by_list_comp(test_arr)

11.1 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [192]:
test_arr = gen_test_arr(0.05, 100)

In [193]:
%%timeit
coords_by_loops(test_arr)

738 µs ± 42.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [194]:
%%timeit
coords_by_list_comp(test_arr)

832 µs ± 119 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [195]:
test_arr = gen_test_arr(0.05, 500)

In [196]:
%%timeit
coords_by_loops(test_arr)

21.9 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [197]:
%%timeit
coords_by_list_comp(test_arr)

21.7 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [198]:
test_arr = gen_test_arr(0.05, 1000)

In [199]:
%%timeit
coords_by_loops(test_arr)

111 ms ± 21.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [200]:
%%timeit
coords_by_list_comp(test_arr)

87.9 ms ± 5.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


It appears to be so in 3 out of four cases, perhaps more so as it scales. Still, as any extra speed would generally be welcomed, it probably wouldn't be a bad idea to swap in the list-comp build.

I could also speed up the process by constructing a row of zeros once instead of generating one for each coordinate.

In [17]:
def zero_matrix_v2(arr_in, validate=True):
    """Second version for 2D matrix"""
    
    # edges, checks
    assert len(arr_in) > 0, "Zero length matrix"
    # validate evenness of n-dimension
    if validate:
        n_dim = len(arr_in[0])
        for row in arr_in[1:]:
            assert len(row) == n_dim, "One or more rows are of uneven lengths"
    
    # compile list of zero-coordinates
    zeroes = coords_by_list_comp(arr_in)
    
    # zero out corresponding row/col for each zero value
    if len(zeroes) == 0:
        return arr_in
    else:
        zero_row = [0 for _ in range(len(arr_in[0]))]
        for m_ix, n_ix in zeroes:
            # set column to zeroes
            for m_ix_dyn, _ in enumerate(arr_in):
                arr_in[m_ix_dyn][n_ix] = 0
            # set row to zeroes
            arr_in[m_ix] = zero_row
    
    return arr_in

Do they both work?

In [223]:
test_arr = gen_test_arr(0.05, 6, 8)
zero_matrix(deepcopy(test_arr)) == zero_matrix_v2(deepcopy(test_arr))

True

Let's see how the modifications helped or not. At this point, we'll want to adjust the density of zeros dynamically, as it will quickly turn into an exercise of setting all values to zero, as there will be at least one zero per row and thus per column. Arbitrarily, we'll shoot for one zero every two rows, which will ensure that not all columns/rows include zeros, and also allow for a decent probability of having multiple zeros in some rows/columns

In [211]:
m = 10
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.05


In [212]:
%%timeit
zero_matrix(deepcopy(test_arr))

237 µs ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [213]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

177 µs ± 49.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [214]:
m = 100
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.005


In [215]:
%%timeit
zero_matrix(deepcopy(test_arr))

11.4 ms ± 453 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [216]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

10.9 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [225]:
m = 500
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.001


In [226]:
%%timeit
zero_matrix(deepcopy(test_arr))

473 ms ± 214 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [227]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

277 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [220]:
m = 1000
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.0005


In [221]:
%%timeit
zero_matrix(deepcopy(test_arr))

1.44 s ± 152 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [222]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

1.51 s ± 389 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Version 2 is significantly faster at certain times, but it's difficult to see how any speed advantage is related to size. I suspect that there are patterns in the random generation that are responsible.

The dimension validation step is an O(m) operation, but this contributes the same time for each version.

### Even More Speed?

If there are multiple zeros in any row or column, it only needs to be zeroed out once. The previous functions can be modified to do this in a couple ways:

1. Building list of zero coordinates — if there was already a zero found in a row, one needn't scan further. And if there was already a zero found in a column, that needn't be added either.
2. Leave the list-build as-is, but only zero out unaltered columns/rows

However, I'll take an alternate approach, and instead of keeping track of zero coordinates, simply building sets of columns and rows containing zeros.

Unfortunately, it's impossible to avoid scanning the entire matrix for zeros. If a zero is found in the second position of a row, one needn't scan the rest of the row for the sake of the rows, but it may contain a zero in a column that no other row contains. The only exception to this condition would be if all rows/columns are found to contain zeros; one could break the search loop at this point and simply return a MxN array filled with zeros. This gives it a worst-case complexity of O(m*n).

In [18]:
def zero_matrix_v3(arr_in, validate=True):
    """Third version for 2D matrix"""
    
    # edges, checks
    assert len(arr_in) > 0, "Zero length matrix"
    # validate evenness of n-dimension
    if validate:
        n_dim = len(arr_in[0])
        for row in arr_in[1:]:
            assert len(row) == n_dim, "One or more rows are of uneven lengths"
    
    # compile sets of rows and columns containing zeroes
    z_rows = set()
    z_cols = set()
    for m_ix, row in enumerate(arr_in):
        for n_ix, val in enumerate(row):
            if val == 0:
                z_rows.add(m_ix)
                z_cols.add(n_ix)
    
    # zero out corresponding row/col for each zero value
    if len(z_rows) == 0:
        return arr_in
    else:
        zero_row = [0 for _ in range(len(arr_in[0]))]
        for m_ix in z_rows:
            arr_in[m_ix] = zero_row
        for n_ix in z_cols:
            for m_ix, _ in enumerate(arr_in):
                arr_in[m_ix][n_ix] = 0

    return arr_in

In [302]:
# test against second version
test_arr = gen_test_arr(0.05, 12, 8)
zero_matrix_v2(deepcopy(test_arr)) == zero_matrix_v3(deepcopy(test_arr))

True

Onto some timing...

In [242]:
m = 10
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.05


In [243]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

149 µs ± 9.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [244]:
%%timeit
zero_matrix_v3(deepcopy(test_arr))

188 µs ± 38.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [245]:
m = 100
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.005


In [246]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

10.5 ms ± 385 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [247]:
%%timeit
zero_matrix_v3(deepcopy(test_arr))

11.1 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [248]:
m = 500
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.001


In [249]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

391 ms ± 65.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [250]:
%%timeit
zero_matrix_v3(deepcopy(test_arr))

312 ms ± 47.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [251]:
m = 1000
p = 1/(2*m)
test_arr = gen_test_arr(p, m)
print(f"P(0) = {p}")

P(0) = 0.0005


In [252]:
%%timeit
zero_matrix_v2(deepcopy(test_arr))

1.38 s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [253]:
%%timeit
zero_matrix_v3(deepcopy(test_arr))

1.18 s ± 149 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


V3 is slower for smaller values, perhaps due to the speed of a list comprehension in v2 and the added step of adding values to sets for v3, but faster as it scales. I assume this would be due to the greater occurrence of multiple zeroes per column/row as it scales.

#### A quick check on probability of multiple zeroes per row...

In [303]:
def binomial(n, k, p_suc):
    """Returns probability of k successes in n trials given P(success)"""

    fac = np.math.factorial
    n_choose_k = fac(n) / (fac(n-k)*fac(k))
    return n_choose_k * p_suc**k * (1-p_suc)**(n-k)


def p_more_than_one(n, p):
    """Returns probability of more than one success in n trials given P(success)"""
    
    bin_0 = binomial(m, 0, p)
    bin_1 = binomial(m, 1, p)
    return 1 - bin_1 - bin_0
    

def p_more_than_one_per_row(m):
    """Calculates P(0 > 1x) in one row using binomial distribution.
    Assumes an MxM array with P(0) of one zero per two rows."""
    
    p = 1/(2*m)
    print(f"P(0) = {p}")
    return p_more_than_one(m, p)

In [291]:
for m in [10, 100, 500, 1000]:
    print(f"m = {m}")
    p = p_more_than_one_per_row(m)
    print(f"P(0 > 1x) per row = {p:.4f}")
    print(f"Expected number of rows containing more than one zero: {p*m:.2f}", "\n")

m = 10
P(0) = 0.05
P(0 > 1x) per row = 0.0861
Expected number of rows containing more than one zero: 0.86 

m = 100
P(0) = 0.005
P(0 > 1x) per row = 0.0898
Expected number of rows containing more than one zero: 8.98 

m = 500
P(0) = 0.001
P(0 > 1x) per row = 0.0901
Expected number of rows containing more than one zero: 45.06 

m = 1000
P(0) = 0.0005
P(0 > 1x) per row = 0.0902
Expected number of rows containing more than one zero: 90.17 



Probability of having more than one zero in any given row is very even as m increases, given the dynamic probability of one zero per two rows, but as suspected, the expected number of rows containing multiple zeros increases linearly with the size of each matrix.

### What About With Numpy?

Heretofore, our array has been a list of lists. Replacing a row with zeroes is easy enough, but replacing a column is a slightly cumbersome, iterative process — and it would only become more so as it scaled to N-dimensions. Numpy was designed to make these kinds of operations smooth and fast, so let's see exactly how much smoother/faster it becomes.

In [54]:
test = np.random.choice(2, (4,5), p=[0.05, 0.95])
test

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1]])

In [56]:
z_coords = np.where(test == 0)
z_rows, z_cols = set(z_coords[0]), set(z_coords[1])
print("Rows:", z_rows)
print("Cols:", z_cols)

Rows: {2}
Cols: {2}


In [57]:
test[:, 2] = 0
test[2] = 0
test

array([[1, 1, 0, 1, 1],
       [1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1]])

Much, much easier. Numpifying the final function:

In [35]:
def zero_matrix_np(arr_in, validate=True):
    """Expects Numpy array as input instead of list, and implements Numpy ops"""
    
    # edges, checks
    assert len(arr_in) > 0, "Zero length matrix"
    # validate evenness of n-dimension if list
    # don't need to do this if numpy array
    if validate and isinstance(arr_in, list):
        n_dim = len(arr_in[0])
        for row in arr_in[1:]:
            assert len(row) == n_dim, "One or more rows are of uneven lengths"
    
    # compile sets of rows and columns containing zeroes
    z_coords = np.where(arr_in == 0)
    z_rows, z_cols = set(z_coords[0]), set(z_coords[1])
    
    # zero out corresponding row/col for each zero value
    if len(z_rows) == 0:
        return arr_in
    else:
        for m_ix in z_rows:
            arr_in[m_ix] = 0
        for n_ix in z_cols:
            arr_in[:, n_ix] = 0

    return arr_in

In [38]:
test_arr = np.random.choice(2, (8,12), p=[0.05, 0.95])
test_arr

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]])

In [39]:
zero_matrix_np(test_arr)

array([[1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Success. Let's check some times. I'll bypass the validation step of v3

In [64]:
m = 10
p = 1/(2*m)
test_arr = np.random.choice(2, (m,m), p=[p, 1-p])
print(f"P(0) = {p}")

P(0) = 0.05


In [65]:
%%timeit
zero_matrix_v3(test_arr.copy(), validate=False)

71.1 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [66]:
%%timeit
zero_matrix_np(test_arr.copy())

12.2 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [67]:
m = 100
p = 1/(2*m)
test_arr = np.random.choice(2, (m,m), p=[p, 1-p])
print(f"P(0) = {p}")

P(0) = 0.005


In [68]:
%%timeit
zero_matrix_v3(test_arr.copy(), validate=False)

5.44 ms ± 479 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [69]:
%%timeit
zero_matrix_np(test_arr.copy())

130 µs ± 8.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [70]:
m = 500
p = 1/(2*m)
test_arr = np.random.choice(2, (m,m), p=[p, 1-p])
print(f"P(0) = {p}")

P(0) = 0.001


In [71]:
%%timeit
zero_matrix_v3(test_arr.copy(), validate=False)

127 ms ± 8.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
%%timeit
zero_matrix_np(test_arr.copy())

2 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [73]:
m = 1000
p = 1/(2*m)
test_arr = np.random.choice(2, (m,m), p=[p, 1-p])
print(f"P(0) = {p}")

P(0) = 0.0005


In [74]:
%%timeit
zero_matrix_v3(test_arr.copy(), validate=False)

501 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
%%timeit
zero_matrix_np(test_arr.copy())

11.3 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [85]:
127/2

63.5

#### No contest. Numpy is anywhere from 12x to 63x faster, and seemingly more so as it scales. Not surprising...